In [ ]:
# === IMPORT LIBRARIES ===
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (mean_absolute_error, 
                           mean_squared_error, 
                           r2_score)
import matplotlib.pyplot as plt
%matplotlib inline

# === LOAD CLEANED DATA ===
insurance_df = pd.read_csv('../data/insurance_cleaned.csv')

# === 1. FEATURE ENGINEERING ===
# Create new features
insurance_df['age_group'] = pd.cut(insurance_df['age'], 
                                  bins=[0,30,45,60,100],
                                  labels=['young','middle_aged','senior','elderly'])

insurance_df['bmi_category'] = pd.cut(insurance_df['bmi'],
                                    bins=[0,18.5,25,30,100],
                                    labels=['underweight','normal','overweight','obese'])

# === 2. DATA PREPARATION ===
# Select features and target
features = ['age', 'bmi', 'children', 'smoker', 'region', 'age_group', 'bmi_category']
X = insurance_df[features]
y = insurance_df['charges']

# Convert categorical variables
X = pd.get_dummies(X, columns=['smoker', 'region', 'age_group', 'bmi_category'], drop_first=True)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numeric features
scaler = StandardScaler()
num_cols = ['age', 'bmi', 'children']
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

# === 3. MODEL TRAINING ===
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42)
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    results[name] = {
        'MAE': mean_absolute_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R2': r2_score(y_test, y_pred)
    }

# === 4. MODEL EVALUATION ===
# Print results
results_df = pd.DataFrame(results).T
print("=== Model Performance ===")
display(results_df)

# Feature importance (for Random Forest)
if "Random Forest" in models:
    plt.figure(figsize=(10,6))
    importances = models["Random Forest"].feature_importances_
    feat_importances = pd.Series(importances, index=X.columns)
    feat_importances.nlargest(10).plot(kind='barh')
    plt.title('Feature Importance (Random Forest)')
    plt.show()

# === 5. PREDICTION VISUALIZATION ===
# Compare actual vs predicted values
plt.figure(figsize=(10,6))
plt.scatter(y_test, models["Random Forest"].predict(X_test), alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.xlabel('Actual Charges')
plt.ylabel('Predicted Charges')
plt.title('Actual vs Predicted Medical Charges')
plt.show()